# Sentiment Analysis - Evaluation

## Setup

Firstly, set up the path to the (preprocessed) dataset

In [1]:
# Path to the preprocessed data
import os

fileDir = os.path.dirname(os.path.realpath('__file__'))
absFilePathToPreprocessedDataset = os.path.join(fileDir, '../Data/training.1600000.processed.noemoticon_preprocessed.csv')
pathToPreprocessedDataset = os.path.abspath(os.path.realpath(absFilePathToPreprocessedDataset))
print (pathToPreprocessedDataset)

c:\Petnica\PetnicaNLPWorkshop\Data\training.1600000.processed.noemoticon_preprocessed.csv


Choose the device to run the training on:

In [2]:
device = "cpu"

Set the learning rate parameter:

In [3]:
learningRate = 0.001

## Initialization

In [4]:
import torch.nn as nn
import torch.optim as optim
from TwitterDataset import TwitterDataset
from ModelPerceptron import SentimentClassifierPerceptron

# Step #1: Instantiate the dataset
# instantiate the dataset
dataset = TwitterDataset.load_dataset_and_make_vectorizer(pathToPreprocessedDataset)
# get the vectorizer
vectorizer = dataset.get_vectorizer()

# Step #2: Instantiate the model
# instantiate the model
model = SentimentClassifierPerceptron(num_features=len(vectorizer.text_vocabulary), output_dim=len(vectorizer.target_vocabulary))
# send model to appropriate device
model = model.to(device)

# Step #3: Instantiate the loss function
loss_func = nn.CrossEntropyLoss()

# Step #4: Instantiate the optimizer
optimizer = optim.Adam(model.parameters(), lr=learningRate)

## Training Loop

In [5]:
from Trainer import Trainer

sentiment_analysis_trainer = Trainer(
    dataset=dataset,
    model=model,
    loss_func=loss_func,
    optimizer=optimizer
)

In [6]:
# setup the chosen number of epochs
num_epochs = 200
# setup the chosen batch size
batch_size = 16

report = sentiment_analysis_trainer.train(num_epochs=num_epochs, batch_size=batch_size, device=device)

## Evaluate the results

In [7]:
def evaluate(split):
    loss, accuracy = sentiment_analysis_trainer.evaluate(split=split, device=device, batch_size=batch_size)

    print("Loss: {:.3f}".format(loss))
    print("Accuracy: {:.3f}".format(accuracy))

#### Training Set

In [8]:
evaluate(split="train")

Loss: 0.583
Accuracy: 0.683


#### Validation Set

In [9]:
evaluate(split="validation")

Loss: 0.611
Accuracy: 0.635


#### Test Set

In [10]:
evaluate(split="test")

Loss: 0.586
Accuracy: 0.667


## Inference and classifying new data points

Let's do inference on the new data. This is another evaluation method to make qualitative judgement about whether the model is working.

In [11]:
import torch

def predict(text, model, vectorizer):
    """
    Predict the sentiment of the tweet

    Args:
        text (str): the text of the tweet
        model (SentimentClassifierPerceptron): the trained model
        vectorizer (TwitterVectorizer): the corresponding vectorizer
    Returns:
        sentiment of the tweet (int), probability of that prediction (float)
    """
    # vectorize the text of the tweet
    vectorized_text = vectorizer.vectorize(text)

    # make a tensor with expected size (1, )
    vectorized_text = torch.Tensor(vectorized_text).view(1, -1)

    # run the model on the vectorized text and apply softmax activation function on the outputs
    result = model(vectorized_text, apply_softmax=True)

    # find the best class as the one with the highest probability
    probability_values, indices = result.max(dim=1)

    # take only value of the indices tensor
    index = indices.item()

    # decode the predicted target index into the sentiment, using target vocabulary
    predicted_target = vectorizer.target_vocabulary.find_index(index)

    # take only value of the probability_values tensor 
    probability_value = probability_values.item()

    return predicted_target, probability_value

Let's try the model on some examples:

In [12]:
text = "This is a good day."

predict(text, model, vectorizer)

(1.0, 0.5481831431388855)

In [13]:
text = "I was very sad yesterday."

predict(text, model, vectorizer)

(0.0, 0.5802960991859436)

In [14]:
text = "This is a book."

predict(text, model, vectorizer)

(0.0, 0.5485750436782837)

### More detailed evaluation on the Test Set

In [15]:
from sklearn.metrics import classification_report, confusion_matrix

# run the model on the tweets from test set 
y_predicted = dataset.test_df.text.apply(lambda x: predict(text=x, model=model, vectorizer=vectorizer)[0])

# compare that with labels
print(classification_report(y_true=dataset.test_df.target, y_pred=y_predicted))

# plot confusion matrix
print("Consfusion matrix:")
print(confusion_matrix(y_true=dataset.test_df.target, y_pred=y_predicted))

precision    recall  f1-score   support

         0.0       0.63      0.66      0.65        47
         1.0       0.69      0.66      0.67        53

    accuracy                           0.66       100
   macro avg       0.66      0.66      0.66       100
weighted avg       0.66      0.66      0.66       100

Consfusion matrix:
[[31 16]
 [18 35]]


## Inspecting model weights

In [16]:
fc1_weights = model.fc1.weight.detach()[0]

fc1_weights

tensor([ 0.0112,  0.2577, -0.1487,  0.1158, -0.2133,  0.2401,  0.0293, -0.4294,
        -0.2259,  0.2441,  0.1021, -0.0997, -0.0358,  0.0919,  0.1688,  0.5477,
        -0.1074,  0.2740, -0.3591,  0.3816,  0.7429,  0.0837, -0.4510, -0.3665,
        -0.2547,  0.0701,  0.0649,  0.4849, -0.1010,  0.1412, -0.3777,  0.0610,
         0.6400,  0.1233, -0.1900, -0.2555,  0.1284, -0.3158,  0.1627,  0.6309,
        -0.1842,  0.4015,  0.7346,  0.0987, -0.0600,  0.1897,  0.0705, -0.0974,
        -0.1526,  0.0404, -0.1333, -0.2396,  0.1236, -0.3804,  0.2365, -0.0651,
         0.0427, -0.0227,  0.6912, -0.0187,  0.0571,  0.2057,  0.3370, -0.0661,
         0.1534,  0.3430, -0.0386,  0.1598, -0.0829, -0.1074, -0.3581,  0.0551,
        -0.1597, -0.3796,  0.2563, -0.6876, -0.5059,  0.1989,  0.4228, -0.2822,
         0.0307,  0.0772])

In [17]:
import torch

_, indices = torch.sort(fc1_weights, dim=0, descending=True)

indices = indices.numpy().tolist()

#### Top 20 most infuential words in Negative Tweets

In [18]:
for i in range(20):
    print(vectorizer.text_vocabulary.find_index(indices[i]))

go
want
not
can't
had
really
will
out
don't
i
this
got
work
day
we
I
my
no
about
)


#### Top 20 most infuential words in Positive Tweets

In [19]:
indices.reverse()

for i in range(20):
    print(vectorizer.text_vocabulary.find_index(indices[i]))

tomorrow
(
your
one
good
its
an
you
!
with
way
love
back
can
going
,
that
"
time
all
